In [1]:
wd <- dirname(dirname(dirname(dirname(getwd()))))
source(paste0(wd,"/mission_control/treasure_map.R"))
#source(paste0(wd,"/mission_control/hubble_prep.R"))
#source(paste0(wd,"/mission_control/hubble_settings.R"))

In [2]:
library(tidyverse)
library(stringr)
library(data.table)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:

#### Warm-up

In [3]:
list.files(I_DIR)

[1] "biomarkers_AdjTPM.csv"  "biomarkers_RawTPM.csv"  "cpi_check.Rds"         
 [4] "cpi_go.Rds"             "filters_AdjTPM.Rds"     "filters.Rds"           
 [7] "filtersAdjTPM.Rds"      "filtersRawTPM.Rds"      "forest-ready.Rds"      
[10] "multi_forest.png"       "signals_base.csv"       "uni_forest.png"        
[13] "validation-measure.Rds" "validation-summary.Rds"

In [4]:
raw <- fread(paste0(I_DIR, 'signals_base.csv')) 

In [5]:
hartwig_genes <- (
    readRDS(paste0(TMP_DIR,"exhaustive-plots-base.Rds"))
        %>% filter(feature_group == "isofox", model == "os", dataset == "all", covariates == "tissue")
        %>% select(feature, est, p_val, cor_tcell, cor_tgfb, cor_prolif)
        %>% filter( cor_tcell > .5 | cor_tgfb > .4 | cor_prolif > .4)
)
hartwig_genes$gene <- unlist(lapply(hartwig_genes$feature, function(i) str_split(i, "_")[[1]][2]))

In [6]:
vhio_genes <- (
    read.csv("/workspace/projects/immune_biomarkers/vhio_reference/genes_versions/V13-Table 1.csv", 
            header = FALSE, sep = ";")
            %>% transmute(genes = V2)
)
vhio_genes$genes <- toupper(vhio_genes$genes)
vhio_genes$genes <- unlist(lapply( vhio_genes$genes, function(i) strsplit(i, " ")[[1]][1]))
vhio_genes$genes <- gsub("-", ".", vhio_genes$genes )

In [7]:
#vhio_genes

#### Common Genes

In [8]:
common <- vhio_genes$genes[which(vhio_genes$genes %in% hartwig_genes$gene)]

In [10]:
tcell <- (
    hartwig_genes 
        %>% filter( gene %in% common) 
        %>% arrange(desc(cor_tcell)) 
        %>% select(gene, cor_tcell) 
        %>% head(8)
        %>% pull(gene)
)

In [11]:
tgfb <- (
    hartwig_genes 
        %>% filter( gene %in% common) 
        %>% select(gene, cor_tgfb) 
        %>% arrange(desc(cor_tgfb)) 
        %>% head(8)
        %>% pull(gene)
)

In [12]:
prolif <- (
hartwig_genes 
    %>% filter( gene %in% common, cor_prolif > .5) 
    %>% select(gene, cor_prolif) 
    %>% arrange(desc(cor_prolif)) 
    %>% head(8)
    %>% pull(gene)
)